In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated


Other Join's you might need

In [ ]:
%%bigquery --project august-bucksaw-330517
with took_offer as
    (select 1 customer_id, 34.99 spend union all
    select 2, 21.99 union all
    select 3, 179.0 union all
    select 4, .99 union all
    select 5, 1299.00),

loyalty_club as
    (select 3 customer_id, 1 status union all
    select 4, 1 union all
    select 8, 2 union all
    select 12, 1 union all
    select 10, 2),

base_table as (select t.customer_id, t.spend, c.customer_id loyalty_customer_id, c.status loyalty_status
from took_offer t
full outer join loyalty_club c on t.customer_id=c.customer_id)

select * from (
    select bt.*, 
    case 
        when customer_id is not null and loyalty_customer_id is null then 'took_offer_not_loyalty'
        when customer_id is not null and loyalty_customer_id is not null then 'took_offer_in_loyalty'
        when customer_id is null and loyalty_customer_id is not null then 'no_offer_in_loyalty'
    end row_table
from base_table bt
) where row_table = 'took_offer_not_loyalty'

,customer_id,spend,loyalty_customer_id,loyalty_status,row_table
0,1,34.99,None,None,took_offer_not_loyalty
1,2,21.99,None,None,took_offer_not_loyalty
2,5,1299.00,None,None,took_offer_not_loyalty


Join's mini project - hourly revenue trends

In [ ]:
%%bigquery --project august-bucksaw-330517
with table_base as (
    SELECT extract (hour from o.order_purchase_timestamp) hour,
    round(sum(payment_value),2) sales,
    sum(sum(payment_value)) over() total_sales

    FROM `jrjames83-1171.sampledata.orders` o
    join `jrjames83-1171.sampledata.order_payments` op on op.order_id=o.order_id
    group by 1),

sales_total as (
    select sum(sales) from table_base)

select *, (select * from sales_total) total_table_sales
from table_base 
order by 1 asc 
limit 6

,hour,sales,total_sales,total_table_sales
0,0,374429.87,16008872.12,16008872.12
1,1,175968.42,16008872.12,16008872.12
2,2,66206.97,16008872.12,16008872.12
3,3,41914.04,16008872.12,16008872.12
4,4,28583.21,16008872.12,16008872.12
5,5,26216.23,16008872.12,16008872.12


In [ ]:
%%bigquery --project august-bucksaw-330517
with table_base as (
    SELECT extract (hour from o.order_purchase_timestamp) hour,
    round(sum(payment_value),2) sales,
    cast(sum(sum(payment_value)) over() as bigint) total_sales

    FROM `jrjames83-1171.sampledata.orders` o
    join `jrjames83-1171.sampledata.order_payments` op on op.order_id=o.order_id
    group by 1),

sales_total as (
    select cast(sum(sales) as bigint) from table_base)

select *, (select * from sales_total) total_table_sales
from table_base 
order by 1 asc 
limit 6

,hour,sales,total_sales,total_table_sales
0,0,374429.87,16008872,16008872
1,1,175968.42,16008872,16008872
2,2,66206.97,16008872,16008872
3,3,41914.04,16008872,16008872
4,4,28583.21,16008872,16008872
5,5,26216.23,16008872,16008872


In [ ]:
%%bigquery --project august-bucksaw-330517
with table_base as (
    SELECT extract (hour from o.order_purchase_timestamp) hour,
    round(sum(payment_value),2) sales,
    cast(sum(sum(payment_value)) over() as bigint) total_sales

    FROM `jrjames83-1171.sampledata.orders` o
    join `jrjames83-1171.sampledata.order_payments` op on op.order_id=o.order_id
    group by 1)

select 
    cast (sum (case when hour between 0 and 5 or hour = 23 then sales else 0 end) as bigint) overnight_sales,
    cast (sum (case when hour between 6 and 11 then sales else 0 end) as bigint) morning_sales,
    cast (sum (case when hour between 12 and 16 then sales else 0 end) as bigint) afternoon_sale
from table_base 

,overnight_sales,morning_sales,afternoon_sale
0,1336677,3541311,5299072


UNNEST, Correlated, Subqueries

In [ ]:
%%bigquery --project august-bucksaw-330517
select GENERATE_ARRAY(1,5) val

,val
0,"[1, 2, 3, 4, 5]"


In [ ]:
%%bigquery --project august-bucksaw-330517
select val 
from unnest(GENERATE_ARRAY(1,5)) val

,val
0,1
1,2
2,3
3,4
4,5


In [ ]:
%%bigquery --project august-bucksaw-330517
select euro, sum(euro) over() sum_euro, index
from unnest(GENERATE_ARRAY(1,5)) euro with OFFSET index

,euro,sum_euro,index
0,1,15,0
1,2,15,1
2,3,15,2
3,4,15,3
4,5,15,4


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (
    select euro, sum(euro) over() sum_euro, index
    from unnest(GENERATE_ARRAY(1,5)) euro with OFFSET index
)
select bt.*, (
    select sum(bt2.euro)
    from base_table bt2
    where bt2.index <= bt.index
) running_euro_sum
from base_table bt

,euro,sum_euro,index,running_euro_sum
0,1,15,0,1
1,2,15,1,3
2,3,15,2,6
3,4,15,3,10
4,5,15,4,15


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (
    select euro, index
    from unnest(GENERATE_ARRAY(1,10)) euro 
    with OFFSET index
)
select *, avg(euro) over (order by index rows BETWEEN 2 preceding AND current row) three_day_ma
    from base_table
limit 6

,euro,index,three_day_ma
0,1,0,1.0
1,2,1,1.5
2,3,2,2.0
3,4,3,3.0
4,5,4,4.0
5,6,5,5.0


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (
    select euro, index
    from unnest(GENERATE_ARRAY(1,10)) euro 
    with OFFSET index
)
select *, lag(euro) over(order by index) my_check
    from base_table
limit 5

,euro,index,my_check
0,1,0,NaN
1,2,1,1.0
2,3,2,2.0
3,4,3,3.0
4,5,4,4.0


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (
    select euro, index
    from unnest(GENERATE_ARRAY(1,10)) euro 
    with OFFSET index
)
select *, euro > lag(euro) over(order by index) my_check
    from base_table
limit 5

,euro,index,my_check
0,1,0,None
1,2,1,True
2,3,2,True
3,4,3,True
4,5,4,True


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (
    select euro, index
    from unnest(GENERATE_ARRAY(1,10)) euro 
    with OFFSET index
)
select distinct my_check from (
    select *, euro > lag(euro) over(order by index) my_check
    from base_table
)
where my_check is not null

,my_check
0,True


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (
    select euro, index
    from unnest(GENERATE_ARRAY(1,10)) euro 
    with OFFSET index
)
select bt.*, bt.euro > (
    select max(bt2.euro)
    from base_table bt2
    where bt.index < bt2.index
) core_sq
from base_table bt

,euro,index,core_sq
0,1,0,False
1,2,1,False
2,3,2,False
3,4,3,False
4,5,4,False
5,6,5,False
6,7,6,False
7,8,7,False
8,9,8,False
9,10,9,None


Customer Order Summary

In [ ]:
%%bigquery --project august-bucksaw-330517
with customer_first_orders as (
    select * from (
        select p.customer_id, p.amount, p.payment_date,
        row_number() over (partition by p.customer_id order by p.payment_date) order_nth
        from `jrjames83-1171.sampledata.payments` p
        order by 1)
    where order_nth = 1
)

select p2.customer_id,
p.amount first_order_amount,
min(p2.payment_date) first_payment,
sum(p2.amount) total_revenue,
p.amount / sum(p2.amount) first_as_pct_total_rev
from `jrjames83-1171.sampledata.payments`p2 
join customer_first_orders p on p2.customer_id=p.customer_id
group by 1,2
limit 6

,customer_id,first_order_amount,first_payment,total_revenue,first_as_pct_total_rev
0,140,4.99,2007-02-16 03:19:44.996577+00:00,83.82,0.059532
1,6,3.99,2007-02-16 02:10:04.996577+00:00,84.75,0.047080
2,425,6.99,2007-02-21 04:04:18.996577+00:00,98.75,0.070785
3,171,0.99,2007-02-16 09:34:35.996577+00:00,104.75,0.009451
4,444,0.99,2007-02-15 03:21:27.996577+00:00,105.72,0.009364
5,19,2.99,2007-02-19 09:11:25.996577+00:00,98.82,0.030257


In [ ]:
%%bigquery --project august-bucksaw-330517

with customer_first_orders as (
    select * from (
        select p.customer_id, p.amount, p.payment_date,
        row_number() over (partition by p.customer_id order by p.payment_date) order_nth
        from `jrjames83-1171.sampledata.payments` p
        order by 1)
    where order_nth = 1
),

summary_so_far as (
    select p2.customer_id,
    p.amount first_order_amount,
    min(p2.payment_date) first_payment,
    sum(p2.amount) total_revenue,
    p.amount / sum(p2.amount) first_as_pct_total_rev
    from `jrjames83-1171.sampledata.payments`p2 
    join customer_first_orders p on p2.customer_id=p.customer_id
    group by 1,2)

select sf.*, 
    (
        select sum(p3.amount)
        from `jrjames83-1171.sampledata.payments` p3
        where p3.customer_id = sf.customer_id
        and date(p3.payment_date) between date(sf.first_payment) and date(sf.first_payment) + 30
    ) customer_LTV_first_30,
    (
        select sum(p3.amount)
        from `jrjames83-1171.sampledata.payments` p3
        where p3.customer_id = sf.customer_id
        and date(p3.payment_date) between date(sf.first_payment) and date(sf.first_payment) + 60
    ) customer_LTV_first_60
from summary_so_far sf
limit 8

,customer_id,first_order_amount,first_payment,total_revenue,first_as_pct_total_rev,customer_LTV_first_30,customer_LTV_first_60
0,443,2.99,2007-02-19 22:56:15.996577+00:00,93.79,0.031880,31.92,55.86
1,533,5.99,2007-02-15 16:25:30.996577+00:00,132.70,0.045139,48.89,98.79
2,410,2.99,2007-02-15 21:26:00.996577+00:00,167.62,0.017838,56.88,140.69
3,158,0.99,2007-02-15 13:41:36.996577+00:00,115.70,0.008557,39.91,95.78
4,281,2.99,2007-03-20 06:03:08.996577+00:00,32.90,0.090881,19.94,32.90
5,424,4.99,2007-02-20 11:07:15.996577+00:00,109.71,0.045484,39.89,66.81
6,261,2.99,2007-02-16 16:17:03.996577+00:00,90.78,0.032937,37.90,75.81
7,537,2.99,2007-02-15 17:38:33.996577+00:00,98.76,0.030275,23.96,72.82


In [ ]:
%%bigquery --project august-bucksaw-330517
with customer_first_orders as (
    select * from (
        select p.customer_id, p.amount, p.payment_date,
        row_number() over (partition by p.customer_id order by p.payment_date) order_nth
        from `jrjames83-1171.sampledata.payments` p
        order by 1)
    where order_nth = 1
),

summary_so_far as (
    select p2.customer_id,
    p.amount first_order_amount,
    min(p2.payment_date) first_payment,
    sum(p2.amount) total_revenue,
    p.amount / sum(p2.amount) first_as_pct_total_rev
    from `jrjames83-1171.sampledata.payments`p2 
    join customer_first_orders p on p2.customer_id=p.customer_id
    group by 1,2)

select sf.*, 
    (
        select sum(p3.amount)
        from `jrjames83-1171.sampledata.payments` p3
        where p3.customer_id = sf.customer_id
        and date(p3.payment_date) between date(sf.first_payment) and 
        date_add(date(sf.first_payment), interval 30 day)
    ) customer_LTV_first_30,
    (
        select sum(p3.amount)
        from `jrjames83-1171.sampledata.payments` p3
        where p3.customer_id = sf.customer_id
        and date(p3.payment_date) between date(sf.first_payment) and date(sf.first_payment) + 60
    ) customer_LTV_first_60,
    date_add(date(sf.first_payment), interval 30 day) thirty_days_from_first_order
from summary_so_far sf
limit 8

,customer_id,first_order_amount,first_payment,total_revenue,first_as_pct_total_rev,customer_LTV_first_30,customer_LTV_first_60,thirty_days_from_first_order
0,2,2.99,2007-02-17 19:23:24.996577+00:00,123.74,0.024164,26.92,57.86,2007-03-19
1,25,4.99,2007-02-15 10:46:00.996577+00:00,104.73,0.047646,46.90,82.79,2007-03-17
2,27,4.99,2007-02-15 08:40:08.996577+00:00,123.70,0.040340,66.86,101.75,2007-03-17
3,554,4.99,2007-02-17 07:22:36.996577+00:00,95.80,0.052088,62.89,95.80,2007-03-19
4,587,2.99,2007-02-15 09:57:43.996577+00:00,102.76,0.029097,36.91,68.82,2007-03-17
5,23,1.99,2007-02-19 15:13:01.996577+00:00,106.73,0.018645,42.87,64.81,2007-03-21
6,118,4.99,2007-02-16 16:28:03.996577+00:00,76.80,0.064974,13.96,44.87,2007-03-18
7,85,1.99,2007-02-16 10:35:23.996577+00:00,72.79,0.027339,40.89,55.83,2007-03-18
